<a href="https://colab.research.google.com/github/sugarforever/wtf-langchain/blob/main/09_Callbacks/09_Callbacks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LangChain Callback 示例

## 准备环境

In [ ]:
pip install langchain==0.3.7
pip install langchain-chroma==0.1.4
pip install langchain-community==0.3.5
pip install langchain-core==0.3.18
pip install langchain-huggingface==0.1.2
pip install langchain-ollama==0.2.0
pip install langchain-openai==0.2.8
pip install langchain-text-splitters==0.3.2

## 示例代码

1. 内置回调处理器 `StdOutCallbackHandler`

In [13]:
from langchain_core.callbacks import StdOutCallbackHandler
from langchain.chains import LLMChain
from langchain_ollama import OllamaLLM
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

handler = StdOutCallbackHandler()
llm = OllamaLLM(temperature=0, model="qwen2.5:14b", callbacks=[handler])
template = "Who is {name}?"
prompt = PromptTemplate(template=template, input_variables=['name'])
# chain = LLMChain(llm=llm, prompt=prompt, callbacks=[handler])
# chain.invoke("Super Mario")
chain = prompt | llm | StrOutputParser()
chain.invoke("Super Mario")

'Super Mario is one of the most iconic video game characters in history, created by Nintendo. He first appeared in 1985 as the protagonist of "Super Mario Bros." for the Nintendo Entertainment System (NES). The character was originally named "Jumpman" but was later renamed to Mario after the Italian plumber Mr. Video Game designer Shigeru Miyamoto chose the name.\n\nMario is typically depicted wearing a red hat and overalls with blue suspenders, and he has brown hair and a bushy moustache. He\'s known for his ability to jump very high and far, which helps him rescue Princess Peach from the clutches of Bowser, the main antagonist in many Mario games. Over time, Super Mario has appeared in numerous video games across various Nintendo platforms, including handheld devices like the Game Boy and more recent consoles such as the Switch.\n\nIn addition to platformers, Mario also stars in other game genres, such as racing (Mario Kart series), sports (Mario Tennis, Mario Golf), puzzle-solving (

2. 自定义回调处理器

我们来实现一个处理器，统计每次 `LLM` 交互的处理时间。

In [10]:
from langchain_core.callbacks.base import BaseCallbackHandler
import time

class TimerHandler(BaseCallbackHandler):

    def __init__(self) -> None:
        super().__init__()
        self.previous_ms = None
        self.durations = []

    def current_ms(self):
        return int(time.time() * 1000 + time.perf_counter() % 1 * 1000)

    def on_chain_start(self, serialized, inputs, **kwargs) -> None:
        self.previous_ms = self.current_ms()

    def on_chain_end(self, outputs, **kwargs) -> None:
        if self.previous_ms:
          duration = self.current_ms() - self.previous_ms
          self.durations.append(duration)

    def on_llm_start(self, serialized, prompts, **kwargs) -> None:
        self.previous_ms = self.current_ms()

    def on_llm_end(self, response, **kwargs) -> None:
        if self.previous_ms:
          duration = self.current_ms() - self.previous_ms
          self.durations.append(duration)

In [14]:
llm = OllamaLLM(temperature=0, model="qwen2.5:14b", callbacks=[handler])
timerHandler = TimerHandler()
prompt = PromptTemplate.from_template("What is the HEX code of color {color_name}?")
chain = prompt | llm
response = chain.invoke(input="blue")
print(response)
response = chain.invoke(input="purple")
print(response)

timerHandler.durations

 The HEX code for the color blue can vary depending on which specific shade of blue you're referring to, but one common representation of blue in web design and digital contexts is #0000FF or a lighter variant like #ADD8E6. If you are looking for a particular shade of blue, please specify!
 The HEX code for the color purple can vary depending on which specific shade of purple you're referring to, but one common representation of purple in web design and digital contexts is #800080. This is a medium shade of purple often referred to as "purple" or "web purple." If you need a different shade of purple, please specify the tone or provide more details!


[]